<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/LH/lh_ac_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
def sliding_window_transformation(dataset, window_size=10):
  '''
  A function to transform time-series data into tabular by sliding window
  Args:
    dataset: the target time-series data.
    window_size: default size=10
  Returns: the tansformed tabular data
  '''
  dataX = []
  for i in range(len(dataset)-window_size+1):
      a = dataset[i:(i+window_size)]
      dataX.append(a)
  return np.array(dataX)

In [0]:
#uploaded = files.upload()
'''Load data'''
#ac_data = pd.read_csv(io.BytesIO(uploaded['lh_ac.csv']))
ac_data = pd.read_csv('lh_ac.csv')

In [0]:
ac_data.head()

,meter_power,device_power,month,hour,weekday
0,600.00,0.0,2,23,0
1,604.02,0.0,2,23,0
2,608.04,0.0,2,23,0
3,612.06,0.0,2,23,0
4,616.08,0.0,2,23,0


In [0]:
#uploaded = files.upload()
#nn_data = pd.read_csv(io.BytesIO(uploaded['nn_data.csv']))
nn_data = pd.read_csv('nn_data.csv')
print(nn_data.shape)

(426081, 4)


In [0]:
scaler = MinMaxScaler()
nn_data = scaler.fit_transform(nn_data.astype(float))

In [0]:
window_size=60
whe_month = ac_data.month.as_matrix()[window_size-1:].reshape([-1,1])
whe_hour = ac_data.hour.as_matrix()[window_size-1:].reshape([-1,1])
whe_weekday = ac_data.weekday.as_matrix()[window_size-1:].reshape([-1,1])
non_time = np.hstack([whe_month,whe_hour,whe_weekday])
data_arr = np.hstack([nn_data,non_time])

dataX = pd.get_dummies(pd.DataFrame(data_arr),columns=[4,5,6]).values


In [0]:
dataX.shape

(426081, 46)

In [0]:
Y_raw = ac_data['device_power'].values[window_size-1:,].reshape([-1,1])
Y_raw.shape
categorized_Y = np.ones(Y_raw.shape)*3
categorized_Y[[np.where(Y_raw==0)[0]],:]=0
categorized_Y[[np.where((Y_raw>0)&(Y_raw<=100))[0]],:]=1
categorized_Y[[np.where((Y_raw>100)&(Y_raw<=1000))[0]],:]=2
categorized_Y[np.where((Y_raw>1000))[0],:]=3

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
Y_1hot = encoder.fit_transform(categorized_Y)

In [0]:
max_indice = np.argmax(Y_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    242622
1    153865
3     20356
2      9238
dtype: int64

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dataX, Y_1hot, test_size=0.1, shuffle=True)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=1024
unit_num=128

In [0]:
#define the Feedforward Model
'''Feedforward Model Definition'''
tf.reset_default_graph()

''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, 46],name='ff_x')
tf_y = tf.placeholder(tf.int32, [None, 4],name='ff_y')

l1 = tf.layers.dense(tf_x,64,activation=tf.nn.leaky_relu,name='ff_l1')
l2 = tf.layers.dense(l1,128,activation=tf.nn.leaky_relu,name='ff_l2')
l3 = tf.layers.dense(l2,256,activation=tf.nn.leaky_relu,name='ff_l3')
l4 = tf.layers.dense(l3,128,activation=tf.nn.leaky_relu,name='ff_l4')
l5 = tf.layers.dense(l4,64,activation=tf.nn.leaky_relu,name='ff_l5')
l6 = tf.layers.dense(l5,32,activation=tf.nn.leaky_relu,name='ff_l6')
l7 = tf.layers.dense(l6,16,activation=tf.nn.leaky_relu,name='ff_l7')
l8 = tf.layers.dense(l7,8,activation=tf.nn.leaky_relu,name='ff_l8')
pred = tf.layers.dense(l8,4,activation=tf.nn.relu,name='ff_pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:8000,1:8000,2:8000,3:8000})

sess = tf.Session()
sess.run(init_op)
#saver.restore(sess, 'my_net/save_hpe_nn.ckpt')

for i in range(0,10):
  X, y = tl.fit_resample(X_train, y_train)
  whole_train = np.concatenate([X,y],axis=1)
  for j in range(0,50):
    print('Loop'+str(i)+',###iteration: '+str(j)+'###')
    batch_index = np.random.choice(32000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-4].reshape([-1,46])
    batch_y = batch_train[:,-4:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],46]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if(j//99==0):
      save_path = saver.save(sess, "my_net_nn/save_ac_nn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],46]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()


Loop0,###iteration: 0###
train loss= 1.3806, Acc=0.28
Test Acc=0.37
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
[[  3 224  11   0]
 [  0 259   0   0]
 [  1 256  29   0]
 [  0 219  22   0]]
Loop0,###iteration: 1###
train loss= 1.3755, Acc=0.35
Test Acc=0.38
[[  7 211  44   0]
 [  0 251   3   0]
 [  2 149  97   0]
 [  1 168  91   0]]
Loop0,###iteration: 2###
train loss= 1.3687, Acc=0.44
Test Acc=0.37
[[  0 127 107   0]
 [  1 253   5   0]
 [  0  62 199   0]
 [  0  89 181   0]]
Loop0,###iteration: 3###
train loss= 1.3639, Acc=0.43
Test Acc=0.36
[[  3 117 152   0]
 [  0 234  11   0]
 [  0  46 208   0]
 [  0  49 204   0]]
Loop0,###iteration: 4###
train loss= 1.3543, Acc=0.44
Test Acc=0.36
[[  0  98 150   0]
 [  0 244   9   0]
 [  0  54 203   0]
 [  0  50 216   0]]
Loop0,###iteration: 5###
train loss= 1.3465, Acc=0.42
Test Acc=0.36
[[  0 125 160   0]
 [  0 212  13   0]
 [  0  44 221   0]
 [  0  61 188   0]]
Loop0,###iteration: 6###
train los